# In this notebook, we will be understanding the different metrics for computer vision tasks and subtasks.
1. obejct detection bbox
2. semantic segmentation
3. instance segmentation
4. pose/keypoint detection
5. panoptic segmentation


# Metrics used in standardize test
1. pycocotools (coco RLE)

# Torch

In [ ]:
# pytorch lib
import torch

# tensorflow lib
import tensorflow as tf

# computer vision
import cv2

# numeric lib
import numpy as np

# python lib
from collections import Counter

## Intersection over union (IOU)
1. For bbox, the intersection
    - x1 = max(box1[0], box2[0])
    - x2 = min(box1[2], box[2])
    - y1 = max(box1[1], box2[1])
    - y2 = max(box1[3], box3[3])

In [ ]:
def IOU_torch(box_preds, box_labels, format="xyxy"):
    '''
    Calculates IOU for bboxes
    '''
    if format == "xyxy":
        bbox1_x1 = box_preds[..., 0:1] # shape = (n, 1)
        bbox1_y1 = box_preds[..., 1:2]
        bbox1_x2 = box_preds[..., 2:3]
        bbox1_y2 = box_preds[..., 3:4]
        bbox2_x1 = box_labels[..., 0:1] 
        bbox2_y1 = box_labels[..., 1:2]
        bbox2_x2 = box_labels[..., 2:3]
        bbox2_y2 = box_labels[..., 3:4]

    elif format == "xywh": #yoloV5 format
        bbox1_x1 = box_preds[..., 0:1] - box_preds[..., 2:3] / 2
        bbox1_y1 = box_preds[..., 1:2] - box_preds[..., 3:4] / 2
        bbox1_x2 = box_preds[..., 0:1] + box_preds[..., 2:3] / 2
        bbox1_y2 = box_preds[..., 1:2] + box_preds[..., 3:4] / 2

        bbox2_x1 = box_labels[..., 0:1] - box_labels[..., 2:3] / 2
        bbox2_y1 = box_labels[..., 1:2] - box_labels[..., 3:4] / 2
        bbox2_x2 = box_labels[..., 0:1] + box_labels[..., 2:3] / 2
        bbox2_y2 = box_labels[..., 1:2] + box_labels[..., 3:4] / 2

    else:
        raise NotImplementedError("This bbox format is not supported")

    bbox1_area =  (bbox1_x2-bbox1_x1) * (bbox1_y2-bbox1_y1)
    bbox2_area =  (bbox2_x2-bbox2_x1) * (bbox2_y2-bbox2_y1)
    
    intersection_x1 = torch.max(bbox1_x1,bbox2_x1)
    intersection_y1 = torch.max(bbox1_y1,bbox2_y1)
    intersection_x2 = torch.min(bbox1_x2,bbox2_x2)
    intersection_y2 = torch.min(bbox1_y2,bbox2_y2)

    # if the box dont intersect, one or more of the difference will be negative
    intersection_area = torch.clamp(intersection_x2-intersection_x1, min=0) * torch.clamp(intersection_y2-intersection_y1, min=0) 

    return intersection_area / (bbox1_area + bbox2_area- intersection_area + 1e-7) #epilson for stability

In [ ]:
# test_1 base case xyxy format
torch_box_1 = torch.tensor([2,2,6,6])
torch_box_2 = torch.tensor([4,4,7,8])
assert torch.abs(IOU_torch(torch_box_1,torch_box_2)-torch.tensor([4/24]))< 1e-6, "Test 1 Wrong!"

# test 2 no intersection xyxy format
torch_box_3 = torch.tensor([2,2,4,4])
torch_box_4 = torch.tensor([4,4,7,8])
assert IOU_torch(torch_box_3,torch_box_4) < torch.tensor([1e-6]), "Test 2 Wrong!"

# test 3 batch input
torch_group_box1 = torch.tensor(
        [
            [0, 0, 2, 2],
            [0, 0, 2, 2],
            [0, 0, 2, 2],
            [0, 0, 2, 2],
            [0, 0, 2, 2],
            [0, 0, 3, 2],
        ]
    )
torch_group_box2 = torch.tensor(
        [
            [3, 0, 5, 2],
            [3, 0, 5, 2],
            [0, 3, 2, 5],
            [2, 0, 5, 2],
            [1, 1, 3, 3],
            [1, 1, 3, 3],
        ]
    )

assert torch.all(torch.abs(IOU_torch(torch_group_box1,torch_group_box2)-torch.Tensor([0, 0, 0, 0, 1 / 7, 0.25]).reshape(-1,1)) < torch.tensor(([1e-6]*6)).reshape(-1,1)), "Group test is Wrong!"

# test 4 base case xywh format
torch_box_5 = torch.tensor([0.8, 0.1, 0.2, 0.2])
torch_box_6 = torch.tensor([0.9, 0.2, 0.2, 0.2])
assert torch.abs(IOU_torch(torch_box_5,torch_box_6, format="xywh")-torch.tensor([1/7])) < 1e-6, "Test 3 Wrong!"

# test 5 no intersection xywh format
torch_box_7 = torch.tensor([0.25, 0.15, 0.3, 0.1])
torch_box_8 = torch.tensor([0.25, 0.35, 0.3, 0.1])
assert IOU_torch(torch_box_7,torch_box_8, format="xywh") < torch.tensor([1e-6]), "Test 4 Wrong!"

## Mean average precision (mAP@ IOU threshold, 0.5,0.55...)

1. Get all the bounding box predictions 
2. Sort by descending confidence score
3. calculate precision and recall and plot on curve (PR curve)
4. Calculate area under curve
5. Repeat for all classes


Note:
1. precision = TP/All_detection
2. recall = TP/All_ground_truth

In [ ]:
def torch_mean_average_precision(pred_boxes, label_boxes, iou_threshold=0.5, format='xyxy', num_classes=20):
    #pred/label boxes [[train_idx, class_pred, prob_score, x1, y1, x2, y2]...]
    
    average_precisions = []
    epsilon = 1e-6

    for label_class in range(num_classes):
        class_detections = []
        class_ground_truths =[]

        for pred_box in pred_boxes:
            if pred_box[1] == label_class:
                class_detections.append(pred_box)

        for label_box in label_boxes:
            if label_box[1] == label_class:
                class_ground_truths.append(label_box)

        amount_bboxes = Counter([gt[0] for gt in class_ground_truths]) # {train_idx: n_boxes...}

        for key,val in amount_bboxes.items():
            amount_bboxes[key] = torch.zeros(val)
        # {0: [0,0,0,0]} -> keep track of which grount truth bbox are fulfilled

        class_detections.sort(key=lambda x:x[2], reverse=True)
        TP = torch.zeros(len(class_detections))
        FP = torch.zeros(len(class_detections))

        total_true_boxes = len(class_ground_truths)

        for detection_idx, detection in enumerate(class_detections):
            ground_truth_img = [
                                bbox for bbox in class_ground_truths if bbox[0]==detection[0]
            ]

            num_gts =  len(ground_truth_img)
            best_iou = 0

            for idx, gt in enumerate(ground_truth_img):
                iou = IOU_torch(
                    torch.tensor(detection[3:]),
                    torch.tensor(gt[3:]),
                    format = format
                )

                if iou > best_iou:
                    best_iou = iou
                    best_gt_idx = idx

            if best_iou > iou_threshold:
                if amount_bboxes[detection[0]][best_gt_idx] == 0:
                    TP[detection_idx] = 1
                    amount_bboxes[detection[0]][best_gt_idx] =1

                else:
                    FP[detection_idx] = 1

            else:
                FP[detection_idx] = 1

        TP_cum_sum = torch.cumsum(TP, dim=0)
        FP_cum_sum = torch.cumsum(FP, dim=0)
        recalls = TP_cum_sum / (total_true_boxes+epsilon)
        precisions = torch.divide(TP_cum_sum, (TP_cum_sum+FP_cum_sum+epsilon))
        
        precisions = torch.cat((torch.tensor([1]), precisions))
        recalls = torch.cat((torch.tensor([0]), recalls))
        average_precisions.append(torch.trapz(precisions,recalls))
    
    return sum(average_precisions)/len(average_precisions) #average over all classes


In [ ]:
t1_preds = [
        [0, 0, 0.9, 0.55, 0.2, 0.3, 0.2],
        [0, 0, 0.8, 0.35, 0.6, 0.3, 0.2],
        [0, 0, 0.7, 0.8, 0.7, 0.2, 0.2],
]
t1_targets = [
        [0, 0, 0.9, 0.55, 0.2, 0.3, 0.2],
        [0, 0, 0.8, 0.35, 0.6, 0.3, 0.2],
        [0, 0, 0.7, 0.8, 0.7, 0.2, 0.2],
]
assert torch.abs(torch_mean_average_precision(t1_preds,t1_targets, format='xywh', num_classes=1) - torch.tensor(1)) < torch.tensor(1e-6), "test_1 failed"

t3_preds = [
        [0, 1, 0.9, 0.55, 0.2, 0.3, 0.2],
        [0, 1, 0.8, 0.35, 0.6, 0.3, 0.2],
        [0, 1, 0.7, 0.8, 0.7, 0.2, 0.2],
]
t3_targets = [
        [0, 0, 0.9, 0.55, 0.2, 0.3, 0.2],
        [0, 0, 0.8, 0.35, 0.6, 0.3, 0.2],
        [0, 0, 0.7, 0.8, 0.7, 0.2, 0.2],
]
assert torch.abs(torch_mean_average_precision(t3_preds,t3_targets, format='xywh', num_classes=1) - torch.tensor(0)) < torch.tensor(1e-6), "test_2 failed"

t4_preds = [
            [0, 0, 0.9, 0.15, 0.25, 0.1, 0.1],
            [0, 0, 0.8, 0.35, 0.6, 0.3, 0.2],
            [0, 0, 0.7, 0.8, 0.7, 0.2, 0.2],
        ]

t4_targets = [
            [0, 0, 0.9, 0.55, 0.2, 0.3, 0.2],
            [0, 0, 0.8, 0.35, 0.6, 0.3, 0.2],
            [0, 0, 0.7, 0.8, 0.7, 0.2, 0.2],
        ]

assert torch.abs(torch_mean_average_precision(t4_preds,t4_targets, format='xywh', num_classes=1) - torch.tensor(5 / 18)) < torch.tensor(1e-6), "test_3 failed"

## Resources

1. IOU pytorch implementation
    - https://www.youtube.com/watch?v=XXYG5ZWtjj0
2. IOU segmantic segmentation
    - https://www.youtube.com/watch?v=0FmNxqLFeYo&t=1s
3. Mean Average Precision pytorch implementation
    - https://www.youtube.com/watch?v=FppOzcDvaDI
    - https://www.youtube.com/watch?v=c45jSJ3WGds

4. Non maximum supression
    - https://towardsdatascience.com/implementation-of-mean-average-precision-map-with-non-maximum-suppression-f9311eb92522
    - https://www.youtube.com/watch?v=YDkjWEN8jNA
